### Evaluation step2

In [ ]:
# Read 3 lengths result from step1
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join
from tempfile import TemporaryFile
import random

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileName = ['RFE_2days_prediction.csv', 'RFE_5days_prediction.csv', 'RFE_10days_prediction.csv', ]

fileName1 = loadPath + fileName[0]
fileName2 = loadPath + fileName[1]
fileName3 = loadPath + fileName[2]
data_2 = pd.read_csv(fileName1)
data_5 = pd.read_csv(fileName2)
data_10 = pd.read_csv(fileName3)

In [ ]:
# sort by relevant ranking
data_2.sort_values('Rank', ascending = True)

In [ ]:
# Prepare dataset for step 2, use the weekly labeled dataset, this cell is for expanded features

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from time import sleep
from os import listdir
from os.path import isfile, join
    
# We get the result from data exploring
# Extract effective features from the original dataset and save them

def main():
    processedList = []

    loadPath = 'yourloadpath'
    savePath = 'yoursavepath'

    fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

#     fileList = ['002645.SZ.csv']
    
    for f in fileList:
        fileName = loadPath + f
        data = pd.read_csv(fileName)
        idxList = data.index.values.tolist()

        # save all column names that should be done normalization into the list below
        colName = [
                     'ts_code',
                     'trade_date', 
                     'SLOWK_maxmin',
                     'SLOWK',
                     'SLOWD_maxmin',
                     'RSI_5_maxmin',
                     'SLOWD',
                     'RSI_5',
                     'SLOWK_flc',
                     'WNR_9_maxmin',
                     'w_up_down'
                  ]
        try:
            data = data[colName]
            fileName = savePath + f
            data.to_csv(fileName, encoding = 'utf-8', index = False)
        except:
            print('Exception on file {}'.format(f))
            
main()

In [ ]:
# Prepare dataset for step 2, use the weekly labeled dataset, this cell is for original features

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from time import sleep
from os import listdir
from os.path import isfile, join
    
# We get the result from data exploring
# Extract effective features from the original dataset and save them

def main():
    processedList = []

    loadPath = 'yourloadpath'
    savePath = 'yoursavepath'

    fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

#     fileList = ['002645.SZ.csv']
    
    for f in fileList:
        fileName = loadPath + f
        data = pd.read_csv(fileName)
        idxList = data.index.values.tolist()

        # save all column names that should be done normalization into the list below
        colName = [
                     'ts_code',
                     'trade_date', 
                     'SLOWK',
                     'SLOWD',
                     'RSI_5',
                     'w_up_down'
                  ]
        try:
            data = data[colName]
            fileName = savePath + f
            data.to_csv(fileName, encoding = 'utf-8', index = False)
        except:
            print('Exception on file {}'.format(f))
            
main()

In [ ]:
# Test the extended feature set using simple NN
# Step 1, training dataset

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from time import sleep
from os import listdir
from os.path import isfile, join
import random

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

fileList = fileList[-1000:]

fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, 8])
y = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    y1 = data['w_up_down'].values
    X1 = data.iloc[:,2:10].copy() # To avoid the case where changing X also changes data
    X1.fillna(0, inplace=True)
    try:
        X1 = scaler.fit_transform(X1)
        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

In [ ]:
# Test the extended feature set using simple NN
# Step 2, test dataset

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = fileList[:2000]

fileList = random.sample(fileList, 100)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, 8])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:10].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    try:
        X1_test = scaler.fit_transform(X1_test)
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# create model
model = Sequential()
model.add(Dense(10, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=10, batch_size=50)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
# Test the original feature set using simple NN
# Step 1, training dataset

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from time import sleep
from os import listdir
from os.path import isfile, join
import random

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]

fileList = fileList[-1000:]

fileList = random.sample(fileList, 500)

# fileList = ['002645.SZ.csv', '603876.SH.csv', '002638.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X = np.zeros([1, 3])
y = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    y1 = data['w_up_down'].values
    X1 = data.iloc[:,2:5].copy() # To avoid the case where changing X also changes data
    X1.fillna(0, inplace=True)
    try:
        X1 = scaler.fit_transform(X1)
        y0 = np.zeros([1,1])
        y1 = np.append(y1, y0)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1 = np.delete(y1, (0), axis=0) # delete first row to ensure the lenth is correct

        X = np.vstack((X, X1))
        y = np.append(y, y1)  # stack all labels

        X = X[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y = y[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X = np.delete(X, (0), axis=0) # delete first row
y = np.delete(y, (0), axis=0) # delete first row

In [ ]:
# Test the extended feature set using simple NN
# Step 2, test dataset

import pandas as pd
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from time import sleep
from os import listdir
from os.path import isfile, join

processedList = []

loadPath = 'yourloadpath'
savePath = 'yoursavepath'

fileList = [f for f in listdir(loadPath) if isfile(join(loadPath, f))]
fileList = fileList[:2000]

fileList = random.sample(fileList, 100)

# fileList = ['002169.SZ.csv']

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = np.zeros([1, 3])
y_test = np.zeros([1, 1])

for f in fileList:
    fileName = loadPath + f
    data = pd.read_csv(fileName)
    
    try:
        y1_test = data['w_up_down'].values.tolist()
    except:
        print(f)
    X1_test = data.iloc[:,2:5].copy() # To avoid the case where changing X also changes data, 2 to 27
    X1_test.fillna(0, inplace=True)
    try:
        X1_test = scaler.fit_transform(X1_test)
        y0_test = np.zeros([1,1])
        y1_test = np.append(y1_test, y0_test)  # add a 0 at the end of label and pop all label up one day, use today data to predict tomorrow result
        y1_test = np.delete(y1_test, (0), axis=0) # delete first row to ensure the lenth is correct

        X_test = np.vstack((X_test, X1_test))
        y_test = np.append(y_test, y1_test)

        X_test = X_test[:-1, :]  # delete the last row of X because the last row of indices are correspoinding to y0
        y_test = y_test[:-1]  # delete the last row of y because the last row of label is y0
    except:
        print(f)
    
X_test = np.delete(X_test, (0), axis=0) # delete first row
y_test = np.delete(y_test, (0), axis=0) # delete first row

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define nn

# create model
model = Sequential()
model.add(Dense(10, input_dim=3, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=10, batch_size=50)  # default 150 epochs

# evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
y_pred = model.predict_classes(X_test, batch_size=10, verbose=0)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
classes_label = ['Up','Down']
 
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes_label, normalize=True,
                      title='Normalized confusion matrix')

plt.show()